In [21]:
import pandas as pd

import requests

import os

In [4]:
import tweepy

In [5]:
auth = tweepy.AppAuthHandler('YOUR_CONSUMER_KEY','YOUR_CONSUMER_SECRET')

In [49]:
api = tweepy.API(auth)
output = []
for tweet in tweepy.Cursor(api.search, q='wildfires-filter:retweets',geocode='37.77986,-122.42905,2500km',until='2020-10-26').items(100):
    output.append({'date':tweet.created_at,'text':tweet.text,'geocode':tweet.place})
output = pd.DataFrame(output)

In [51]:
output.isnull().sum()

date        0
text        0
geocode    96
dtype: int64

In [52]:
final = pd.DataFrame(output)

In [53]:
final = pd.concat([final,output])

In [54]:
final

date                                               text  \
0  2020-10-25 23:58:59  @Geno1955 Same here. I haven't shut my down on...   
1  2020-10-25 23:57:08  @fakegingerbitch Ordinarily I would agree.\n\n...   
2  2020-10-25 23:50:38  Shasta County has already seen three relativel...   
3  2020-10-25 23:50:30  Colorado photo dump plus my car setup for my f...   
4  2020-10-25 23:49:05  @IdiosyncraticXL @Nightinghawk Dear Phil, I wi...   
..                 ...                                                ...   
95 2020-10-25 20:07:18  @ContraPoints We’re about to evacuate some mor...   
96 2020-10-25 20:03:02  Climate change is absolutely a Washington Stat...   
97 2020-10-25 20:02:37  COVID and the wildfires here shook me more tha...   
98 2020-10-25 20:00:41  California faces highest fire risk of 2020 wit...   
99 2020-10-25 20:00:23  The Point Fire has grown to 200 acres in the t...   

   geocode  
0     None  
1     None  
2     None  
3     None  
4     None  
..     ...  
95    None  
96    None  
97    None  
98    None  
99    None  

[200 rows x 3 columns]

In [89]:
import time

In [90]:
my_dates = ['2020-10-27','2020-10-26','2020-10-25','2020-10-24','2020-10-23','2020-10-22','2020-10-21','2020-10-20','2020-10-19']

In [191]:
def get_tweets(geocodee,date_range):
    api = tweepy.API(auth)
    output = []
    last = pd.DataFrame(columns=['date','text','geocode'])
    for date in date_range:
        for tweet in tweepy.Cursor(api.search, q='wildfires-filter:retweets',geocode=geocodee,until=date,tweet_mode='extended',lang='en').items(100):
            output.append({'date': tweet.created_at,'text':tweet.full_text,'geocode':tweet.place,'user_location':tweet.user.location})
        time.sleep(5)
    output = pd.DataFrame(output)
        
    return output

In [207]:
la_geo = '34.05349,-118.24532,200km'
sf_geo = '37.77986,-122.42905,300km'

In [198]:
la_4 = get_tweets(la_geo,my_dates)

In [203]:
la_4

date                                               text  \
0   2020-10-26 17:40:11  LATEST: After first reporting that the #Silver...   
1   2020-10-26 17:38:59  Planned power outages affect hundreds of thous...   
2   2020-10-26 17:37:07  When writing political stories you often have ...   
3   2020-10-26 17:36:15  "Planned power outages affect hundreds of thou...   
4   2020-10-26 17:32:54  If you are in a safe place after evacuating fr...   
..                  ...                                                ...   
795 2020-10-18 22:51:06  Despite years of neglect and mismanagement in ...   
796 2020-10-18 22:48:59  COVID - my gym has now been closed 31 weeks \n...   
797 2020-10-18 22:30:06  Wildfires and floods threaten tens of millions...   
798 2020-10-18 22:26:30  @lexis_dawson23 ask him about the type of worl...   
799 2020-10-18 21:47:42  @NBC I think this warrants a journalistic inve...   

    geocode          user_location  
0      None        Los Angeles, CA  
1      None           Altadena, CA  
2      None        Los Angeles, CA  
3      None        Los Angeles, CA  
4      None        Los Angeles, CA  
..      ...                    ...  
795    None        Los Angeles, CA  
796    None      Santa Clarita, CA  
797    None  San Diego, California  
798    None            Los Angeles  
799    None        Los Angeles, CA  

[800 rows x 4 columns]

In [193]:
la_3F = get_tweets(la_geo,my_dates)

In [196]:
la_3F['text'][1]

'Planned power outages affect hundreds of thousands as intense wildfires rage on in Northern California https://t.co/zjs7Z5dMjf'

In [197]:
sf_3F = get_tweets(sf_geo,my_dates)

In [ ]:
sf_3F['text'][0]

In [205]:
best = pd.concat([la_3F,sf_3F])
best.reset_index(inplace=True,drop=True)

In [208]:
best.to_csv('twitter/best_tweets.csv',index=False)

## Next Steps:

- Remove URL's from text.
- Get the geocodes from my User Location for plotting later.

## Problems Encountered:

- It took a lot of tinkering before I was able to search for wildfires in a specific range. To do this I needed to enter the latitidue,longitude,and range of kms or mi.
- Many tweets did not include a Geotag which is why I pulled in User Location to be used as our geocode input later.
- We can only get tweets from the last 7 days. I would need to pay for Premium to get access to the entire database. I will continue to collect more tweets from smaller geo ranges and from today until Thursday. 
- Initially, tweets returned would contain only part of the tweet. I needed to specify in my query that I wanted extended tweets. After doing this, I was able to obtain full tweets.
- I created my function to make running through the entire date range more efficient. Initially, I was searching by individual date and needing to combine dataframes after each run. Very inefficient.
